In [1]:
import pandas as pd
import numpy as np
import shutil
import keras
import os

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.applications.inception_v3 import InceptionV3
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [2]:
NUM_CLASSES = 7
SHORT_EPOCH = 3
LONG_EPOCH = 50
IMAGE_HEIGHT, IMAGE_WIDTH = 224, 224
batch_size = 32

In [3]:
df = pd.read_csv("./labels/labels.csv")

In [4]:
data = df.as_matrix()

/home/john/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [5]:
training_size = 7000
np.random.shuffle(data)
training, test = data[:training_size,:], data[training_size:,:]

In [6]:
training_vals = np.delete(training, 0,1)
test_vals = np.delete(test, 0,1)

In [7]:
def switch_demo(argument):
    switcher = {
        0: "MEL",
        1: "NV",
        2: "BCC",
        3: "AKIEC",
        4: "BKL",
        5: "DF",
        6: "VASC",
    }
    return (switcher.get(argument, "Invalid code"))

In [8]:
path, dirs, files = next(os.walk("/home/john/Documents/ISIC2018/data"))
file_count = len(files)

In [9]:
if file_count > 0:
    for i in range(len(training)):
        d_class = switch_demo(training_vals[i].argmax(axis=0))
        shutil.move("/home/john/Documents/skin/data/"+training[i][0]+".jpg", "/home/john/Documents/skin/train/"+str(d_class)+"/"+str(training[i][0])+".jpg")

In [10]:
if file_count > 0:
    for i in range(len(test)):
        d_class = switch_demo(test_vals[i].argmax(axis=0))
        shutil.move("/home/john/Documents/skin/data/"+test[i][0]+".jpg", "/home/john/Documents/skin/test/"+str(d_class)+"/"+str(test[i][0])+".jpg")

In [11]:
base_model = InceptionV3(weights='imagenet', include_top=False)

In [12]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [13]:

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './train',
        target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        './test',
        target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
        batch_size=batch_size,
        class_mode='categorical')

Found 7000 images belonging to 7 classes.
Found 3015 images belonging to 7 classes.


In [14]:
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=["accuracy"])

In [15]:
checkpoint = ModelCheckpoint("weights/incepV3_v1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


# Train the model 
model.fit_generator(
train_generator,
epochs = SHORT_EPOCH,
validation_data = validation_generator,
callbacks = [checkpoint, early])

Epoch 1/3
219/219 [==============================] - 148s 674ms/step - loss: 2.1536 - acc: 0.6374 - val_loss: 1.5947 - val_acc: 0.6647

Epoch 00001: val_acc improved from -inf to 0.66468, saving model to weights/incepV3_v1.h5
Epoch 2/3
219/219 [==============================] - 130s 592ms/step - loss: 0.9148 - acc: 0.6897 - val_loss: 1.6476 - val_acc: 0.6640

Epoch 00002: val_acc did not improve from 0.66468
Epoch 3/3
219/219 [==============================] - 130s 594ms/step - loss: 0.8503 - acc: 0.7035 - val_loss: 2.0670 - val_acc: 0.6643

Epoch 00003: val_acc did not improve from 0.66468


In [16]:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy',metrics=["accuracy"])

0 input_1
1 conv2d_1
2 batch_normalization_1
3 activation_1
4 conv2d_2
5 batch_normalization_2
6 activation_2
7 conv2d_3
8 batch_normalization_3
9 activation_3
10 max_pooling2d_1
11 conv2d_4
12 batch_normalization_4
13 activation_4
14 conv2d_5
15 batch_normalization_5
16 activation_5
17 max_pooling2d_2
18 conv2d_9
19 batch_normalization_9
20 activation_9
21 conv2d_7
22 conv2d_10
23 batch_normalization_7
24 batch_normalization_10
25 activation_7
26 activation_10
27 average_pooling2d_1
28 conv2d_6
29 conv2d_8
30 conv2d_11
31 conv2d_12
32 batch_normalization_6
33 batch_normalization_8
34 batch_normalization_11
35 batch_normalization_12
36 activation_6
37 activation_8
38 activation_11
39 activation_12
40 mixed0
41 conv2d_16
42 batch_normalization_16
43 activation_16
44 conv2d_14
45 conv2d_17
46 batch_normalization_14
47 batch_normalization_17
48 activation_14
49 activation_17
50 average_pooling2d_2
51 conv2d_13
52 conv2d_15
53 conv2d_18
54 conv2d_19
55 batch_normalization_13
56 batch_norma

In [17]:
model.fit_generator(
train_generator,
epochs = LONG_EPOCH,
validation_data = validation_generator,
callbacks = [checkpoint, early])

Epoch 1/50
219/219 [==============================] - 134s 614ms/step - loss: 0.7464 - acc: 0.7361 - val_loss: 1.6371 - val_acc: 0.6637

Epoch 00001: val_acc did not improve from 0.66468
Epoch 2/50
219/219 [==============================] - 128s 586ms/step - loss: 0.7199 - acc: 0.7372 - val_loss: 1.6674 - val_acc: 0.6643

Epoch 00002: val_acc did not improve from 0.66468
Epoch 3/50
219/219 [==============================] - 128s 586ms/step - loss: 0.6999 - acc: 0.7483 - val_loss: 1.6082 - val_acc: 0.6653

Epoch 00003: val_acc improved from 0.66468 to 0.66534, saving model to weights/incepV3_v1.h5
Epoch 4/50
219/219 [==============================] - 127s 582ms/step - loss: 0.6879 - acc: 0.7501 - val_loss: 1.5797 - val_acc: 0.6663

Epoch 00004: val_acc improved from 0.66534 to 0.66633, saving model to weights/incepV3_v1.h5
Epoch 5/50
219/219 [==============================] - 127s 580ms/step - loss: 0.6803 - acc: 0.7561 - val_loss: 1.5977 - val_acc: 0.6657

Epoch 00005: val_acc did not 

219/219 [==============================] - 126s 577ms/step - loss: 0.4506 - acc: 0.8318 - val_loss: 1.4572 - val_acc: 0.6753

Epoch 00042: val_acc did not improve from 0.67529
Epoch 43/50
219/219 [==============================] - 127s 581ms/step - loss: 0.4462 - acc: 0.8367 - val_loss: 1.4715 - val_acc: 0.6756

Epoch 00043: val_acc improved from 0.67529 to 0.67562, saving model to weights/incepV3_v1.h5
Epoch 44/50
219/219 [==============================] - 128s 586ms/step - loss: 0.4438 - acc: 0.8403 - val_loss: 1.4429 - val_acc: 0.6753

Epoch 00044: val_acc did not improve from 0.67562
Epoch 45/50
219/219 [==============================] - 128s 586ms/step - loss: 0.4371 - acc: 0.8418 - val_loss: 1.4523 - val_acc: 0.6760

Epoch 00045: val_acc improved from 0.67562 to 0.67595, saving model to weights/incepV3_v1.h5
Epoch 46/50
219/219 [==============================] - 127s 579ms/step - loss: 0.4344 - acc: 0.8375 - val_loss: 1.4841 - val_acc: 0.6753

Epoch 00046: val_acc did not improve